In [1]:
from TTS.tts.configs.bark_config import BarkConfig
from TTS.tts.models.bark import Barkg

ModuleNotFoundError: No module named 'TTS'

In [ ]:
processor = BarkProcessor.from_pretrained("suno/bark")


train_dataset = datasets.load_dataset("")


def preprocess_data(batch):
    audio = batch["audio"]
    text = batch["text"]
    inputs = processor(text, audio, return_tensors="pt", padding=True)
    return inputs

train_dataset = train_dataset.map(preprocess_data)

model = BarkForPreTraining.from_pretrained("suno/bark")

# training params
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

# initialise the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=lambda data: {"input_values": data["input_values"], "labels": data["labels"]},
    train_dataset=train_dataset,
)


trainer.train()